In [104]:
import pandas as pd
import glob
import sys
import glob
sys.path.append('/home/pierre/riken/io')

import reader

In [105]:
pd.options.display.max_colwidth = 999

## Allergens

In [106]:
allergens_df = reader.read_fasta('/home/pierre/riken/data/riken_data/COMPARE_2018.txt').assign(is_allergenic=lambda x: True)

In [107]:
allergens_df.head()

,id,name,description,sequences,is_allergenic
0,accession|2|Putative,accession|2|Putative,accession|2|Putative Blo t 1.02 Manual Entry [Blomia tropicalis],"(M, K, F, L, L, V, A, A, L, C, A, L, V, A, I, G, S, C, K, P, T, R, E, E, I, K, T, F, E, Q, F, K, K, V, F, G, K, V, Y, R, N, A, E, E, E, A, R, R, E, H, H, F, K, E, Q, L, K, W, V, E, E, H, N, G, I, D, G, V, E, Y, A, I, N, E, Y, S, D, M, S, E, Q, E, F, S, F, H, L, S, G, G, G, L, N, F, T, Y, M, K, M, E, ...)",True
1,accession|3|Putative,accession|3|Putative,accession|3|Putative vitellogenin I Manual Entry [Gallus gallus],"(E, I, A, S, Q, I, A, Q, E, D, Q, S, T, C, E, V, S, K, G, D, F, K, T, F, D, R, M, S, F, T, C, S, F, N, K, S, C, N, V, V, V, A, Q, D, C, T, E, H, P, K, F, I, I, T, T, R, K, V, D, H, Q, S, L, S, R, E, V, H, I, N, T, S, S, A, N, I, T, I, C, P, A, A, D, S, S, L, L, V, T, C, N, K, E, S, V, L, S, D, S, G, ...)",True
2,accession|100|Putative,accession|100|Putative,accession|100|Putative Act d 9.0101 Manual Entry [Actinidia deliciosa],"(M, S, W, Q, T, Y, V, D, D, H, L, M, C, E, I, E, G, N, Y, L, T, S, A, A, I, I, G, Q, D, G, S, I, W, A, Q, S, A, S, F, P, Q, F, K, P, E, E, I, T, A, I, M, N, D, F, S, E, P, G, T, L, A, P, T, G, L, Y, L, G, G, T, K, Y, M, V, I, Q, G, E, A, G, A, V, I, R, G, K, K, G, P, G, G, V, T, V, K, K, T, N, Q, A, ...)",True
3,accession|CAA32835.1|Allergen,accession|CAA32835.1|Allergen,accession|CAA32835.1|Allergen beta-lactoglobulin [Bos taurus],"(M, K, C, L, L, L, A, L, A, L, T, C, G, A, Q, A, L, I, V, T, Q, T, M, K, G, L, D, I, Q, K, V, A, G, T, W, Y, S, L, A, M, A, A, S, D, I, S, L, L, D, A, Q, S, A, P, L, R, V, Y, V, E, E, L, K, P, T, P, E, G, D, L, E, I, L, L, Q, K, W, E, N, D, E, C, A, Q, K, K, I, I, A, E, K, T, K, I, P, A, V, F, K, I, ...)",True
4,accession|CAA68720.1|Putative,accession|CAA68720.1|Putative,accession|CAA68720.1|Putative precursor [Canis lupus familiaris],"(M, W, F, L, A, L, C, L, A, M, S, L, G, W, T, G, A, E, P, H, F, Q, P, R, I, I, G, G, R, E, C, L, K, N, S, Q, P, W, Q, V, A, V, Y, H, N, G, E, F, A, C, G, G, V, L, V, N, P, E, W, V, L, T, A, A, H, C, A, N, S, N, C, E, V, W, L, G, R, H, N, L, S, E, S, E, D, E, G, Q, L, V, Q, V, R, K, S, F, I, H, P, L, ...)",True


In [108]:
genre_species = (allergens_df.description.str.extract(r'\[(.*?)\]').iloc[:, 0]
                         .str.lower()
                         .str.split(expand=True).iloc[:, :2])
allergens_df.loc[:, 'genre'], allergens_df.loc[:, 'species'] = genre_species.iloc[:, 0], genre_species.iloc[:, 1]

## Non-allergens

In [109]:
no_allergens_path = glob.glob('/home/pierre/riken/data/riken_data/xlsx/*')
excel_df = pd.concat([pd.read_excel(file) for file in no_allergens_path], ignore_index=True)

aa_sequences_path = glob.glob('/home/pierre/riken/data/riken_data/fasta/*')
seq_df = (pd.concat([reader.read_fasta(file) for file in aa_sequences_path], ignore_index=True)
          .assign(Entry=lambda x: x.name.str.split(pat='|', expand=True)[1]))

In [110]:
seq_df.head(10)

,id,name,description,sequences,Entry
0,sp|U3KRF2|DHQS_ACTCH,sp|U3KRF2|DHQS_ACTCH,"sp|U3KRF2|DHQS_ACTCH 3-dehydroquinate synthase, chloroplastic OS=Actinidia chinensis OX=3625 GN=DHQS PE=1 SV=2","(M, A, A, F, S, L, S, A, K, Q, I, L, S, P, S, T, P, R, P, S, L, S, K, S, T, T, A, D, S, S, L, R, F, R, N, P, P, T, L, S, L, R, R, S, S, L, S, S, S, S, N, V, G, R, T, R, L, L, S, S, S, A, S, S, T, A, P, V, M, D, T, S, P, T, K, A, V, S, S, A, P, T, I, V, D, V, D, L, G, D, R, S, Y, P, I, Y, I, G, S, G, ...)",U3KRF2
1,sp|P85076|PME_ACTDE,sp|P85076|PME_ACTDE,sp|P85076|PME_ACTDE Pectinesterase OS=Actinidia deliciosa OX=3627 PE=1 SV=1,"(T, A, V, T, D, I, V, P, D, V, V, V, A, K, D, G, S, G, N, F, T, T, V, G, A, A, V, A, A, A, K, D, S, S, T, A, R, F, V, I, Y, I, K, E, G, A, Y, F, E, Y, V, D, V, D, K, K, K, T, N, L, M, F, I, G, D, G, I, G, K, T, W, I, K, G, N, R, S, V, V, D, G, W, T, T, F, R, S, S, T, V, A, V, V, G, T, G, F, I, A, R, ...)",P85076
2,sp|O02373|UGDH_DROME,sp|O02373|UGDH_DROME,sp|O02373|UGDH_DROME UDP-glucose 6-dehydrogenase OS=Drosophila melanogaster OX=7227 GN=sgl PE=1 SV=1,"(M, K, V, C, C, I, G, A, G, Y, V, G, G, P, T, C, A, V, M, A, L, K, C, P, D, I, V, I, T, L, V, D, K, S, S, E, R, I, A, Q, W, N, S, D, K, L, P, I, Y, E, P, G, L, D, E, V, V, K, K, C, R, N, V, N, L, F, F, S, T, D, I, E, T, A, I, K, E, A, D, L, I, F, I, S, V, N, T, P, T, K, T, C, G, N, G, K, G, R, A, A, ...)",O02373
3,sp|P00785|ACTN_ACTCH,sp|P00785|ACTN_ACTCH,sp|P00785|ACTN_ACTCH Actinidain OS=Actinidia chinensis OX=3625 PE=1 SV=4,"(M, G, L, P, K, S, F, V, S, M, S, L, L, F, F, S, T, L, L, I, L, S, L, A, F, N, A, K, N, L, T, Q, R, T, N, D, E, V, K, A, M, Y, E, S, W, L, I, K, Y, G, K, S, Y, N, S, L, G, E, W, E, R, R, F, E, I, F, K, E, T, L, R, F, I, D, E, H, N, A, D, T, N, R, S, Y, K, V, G, L, N, Q, F, A, D, L, T, D, E, E, F, R, ...)",P00785
4,sp|A5HII1|ACTN_ACTDE,sp|A5HII1|ACTN_ACTDE,sp|A5HII1|ACTN_ACTDE Actinidain OS=Actinidia deliciosa OX=3627 PE=1 SV=1,"(M, G, L, P, K, S, F, V, S, M, S, L, L, F, F, S, T, L, L, I, L, S, L, A, F, N, A, K, N, L, T, Q, R, T, N, D, E, V, K, A, M, Y, E, S, W, L, I, K, Y, G, K, S, Y, N, S, L, G, E, W, E, R, R, F, E, I, F, K, E, T, L, R, F, I, D, E, H, N, A, D, T, N, R, S, Y, K, V, G, L, N, Q, F, A, D, L, T, D, E, E, F, R, ...)",A5HII1
5,sp|O04015|P5CS_ACTDE,sp|O04015|P5CS_ACTDE,sp|O04015|P5CS_ACTDE Delta-1-pyrroline-5-carboxylate synthase OS=Actinidia deliciosa OX=3627 PE=2 SV=1,"(M, D, A, V, D, S, T, R, A, F, V, K, G, V, K, R, V, I, I, K, V, G, T, A, V, V, T, R, A, D, G, R, L, A, L, G, R, L, G, A, L, C, E, Q, I, H, E, L, N, S, Q, G, F, E, V, I, L, V, T, S, G, A, V, G, V, G, R, Q, R, L, R, Y, R, K, L, V, N, S, S, F, A, D, L, Q, K, P, Q, I, E, L, D, G, K, A, C, A, A, V, G, Q, ...)",O04015
6,sp|Q6TPK4|CYT1_ACTDE,sp|Q6TPK4|CYT1_ACTDE,sp|Q6TPK4|CYT1_ACTDE Cysteine proteinase inhibitor 1 OS=Actinidia deliciosa OX=3627 PE=1 SV=1,"(M, V, P, K, P, L, S, L, L, L, F, L, L, L, A, L, S, A, A, V, V, G, G, R, K, L, V, A, A, G, G, W, R, P, I, E, S, L, N, S, A, E, V, Q, D, V, A, Q, F, A, V, S, E, H, N, K, Q, A, N, D, E, L, Q, Y, Q, S, V, V, R, G, Y, T, Q, V, V, A, G, T, N, Y, R, L, V, I, A, A, K, D, G, A, V, V, G, N, Y, E, A, V, V, W, ...)",Q6TPK4
7,sp|P83326|PMEI_ACTDE,sp|P83326|PMEI_ACTDE,sp|P83326|PMEI_ACTDE Pectinesterase inhibitor OS=Actinidia deliciosa OX=3627 GN=PMEI PE=1 SV=2,"(M, A, F, S, Y, C, S, S, S, L, F, V, S, L, L, L, V, I, L, F, I, S, P, L, S, Q, R, P, S, V, K, A, E, N, H, L, I, S, E, I, C, P, K, T, R, N, P, S, L, C, L, Q, A, L, E, S, D, P, R, S, A, S, K, D, L, K, G, L, G, Q, F, S, I, D, I, A, Q, A, S, A, K, Q, T, S, K, I, I, A, S, L, T, N, Q, A, T, D, P, K, L, K, ...)",P83326
8,sp|P49645|ADH1_APTAU,sp|P49645|ADH1_APTAU,sp|P49645|ADH1_APTAU Alcohol dehydrogenase 1 OS=Apteryx australis OX=8822 GN=ADH1 PE=1 SV=2,"(M, S, T, A, G, K, V, I, K, C, K, A, A, V, L, W, E, P, K, K, P, F, S, I, E, E, V, E, V, A, P, P, K, A, H, E, V, R, I, K, I, L, A, T, G, I, C, R, S, D, D, H, V, I, T, G, A, L, V, R, P, F, P, I, I, L, G, H, E, A, A, G, V, V, E, S,

In [111]:
excel_df.head()

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length
0,P33082,AXX15_SOYBN,reviewed,Auxin-induced protein X15,NaN,Glycine max (Soybean) (Glycine hispida),82
1,P69421,ATP9_SOYBN,reviewed,"ATP synthase subunit 9, mitochondrial (Lipid-binding protein)",ATP9 GlmaxMp72,Glycine max (Soybean) (Glycine hispida),74
2,Q2PMV0,ATPB_SOYBN,reviewed,"ATP synthase subunit beta, chloroplastic (EC 3.6.3.14) (ATP synthase F1 sector subunit beta) (F-ATPase subunit beta)",atpB,Glycine max (Soybean) (Glycine hispida),498
3,Q01915,ATPAM_SOYBN,reviewed,"ATP synthase subunit alpha, mitochondrial",ATPA atp1-1 atp1-2 atp1-3 GlmaxMp39 GlmaxMp55 GlmaxMp78,Glycine max (Soybean) (Glycine hispida),508
4,C6T1K6,CASP3_SOYBN,reviewed,Casparian strip membrane protein 3 (GmCASP3),NaN,Glycine max (Soybean) (Glycine hispida),195


In [112]:
non_allergens_df = pd.merge(seq_df, excel_df, on='Entry').assign(is_allergenic=lambda x: False)

In [113]:
genre_species = non_allergens_df.Organism.str.lower().str.split(expand=True).iloc[:, :2]
non_allergens_df.loc[:, 'genre'], non_allergens_df.loc[:, 'species'] = genre_species.iloc[:, 0], genre_species.iloc[:, 1]

## Merging Data

In [114]:
COLS_TO_KEEP = ['is_allergenic', 'sequences', 'genre', 'species']

In [115]:
non_allergens_df = non_allergens_df.loc[:, COLS_TO_KEEP]
allergens_df = allergens_df.loc[:, COLS_TO_KEEP]

df = pd.concat((allergens_df, non_allergens_df), ignore_index=True).assign(seq_len=lambda x: x.sequences.apply(len))

In [116]:
df.head()

,is_allergenic,sequences,genre,species,seq_len
0,True,"(M, K, F, L, L, V, A, A, L, C, A, L, V, A, I, G, S, C, K, P, T, R, E, E, I, K, T, F, E, Q, F, K, K, V, F, G, K, V, Y, R, N, A, E, E, E, A, R, R, E, H, H, F, K, E, Q, L, K, W, V, E, E, H, N, G, I, D, G, V, E, Y, A, I, N, E, Y, S, D, M, S, E, Q, E, F, S, F, H, L, S, G, G, G, L, N, F, T, Y, M, K, M, E, ...)",blomia,tropicalis,333
1,True,"(E, I, A, S, Q, I, A, Q, E, D, Q, S, T, C, E, V, S, K, G, D, F, K, T, F, D, R, M, S, F, T, C, S, F, N, K, S, C, N, V, V, V, A, Q, D, C, T, E, H, P, K, F, I, I, T, T, R, K, V, D, H, Q, S, L, S, R, E, V, H, I, N, T, S, S, A, N, I, T, I, C, P, A, A, D, S, S, L, L, V, T, C, N, K, E, S, V, L, S, D, S, G, ...)",gallus,gallus,284
2,True,"(M, S, W, Q, T, Y, V, D, D, H, L, M, C, E, I, E, G, N, Y, L, T, S, A, A, I, I, G, Q, D, G, S, I, W, A, Q, S, A, S, F, P, Q, F, K, P, E, E, I, T, A, I, M, N, D, F, S, E, P, G, T, L, A, P, T, G, L, Y, L, G, G, T, K, Y, M, V, I, Q, G, E, A, G, A, V, I, R, G, K, K, G, P, G, G, V, T, V, K, K, T, N, Q, A, ...)",actinidia,deliciosa,109
3,True,"(M, K, C, L, L, L, A, L, A, L, T, C, G, A, Q, A, L, I, V, T, Q, T, M, K, G, L, D, I, Q, K, V, A, G, T, W, Y, S, L, A, M, A, A, S, D, I, S, L, L, D, A, Q, S, A, P, L, R, V, Y, V, E, E, L, K, P, T, P, E, G, D, L, E, I, L, L, Q, K, W, E, N, D, E, C, A, Q, K, K, I, I, A, E, K, T, K, I, P, A, V, F, K, I, ...)",bos,taurus,178
4,True,"(M, W, F, L, A, L, C, L, A, M, S, L, G, W, T, G, A, E, P, H, F, Q, P, R, I, I, G, G, R, E, C, L, K, N, S, Q, P, W, Q, V, A, V, Y, H, N, G, E, F, A, C, G, G, V, L, V, N, P, E, W, V, L, T, A, A, H, C, A, N, S, N, C, E, V, W, L, G, R, H, N, L, S, E, S, E, D, E, G, Q, L, V, Q, V, R, K, S, F, I, H, P, L, ...)",canis,lupus,260


In [117]:
df.to_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx.tsv', sep='\t', index=False)